In [1]:
# NOTEBOOK IMPORTS
import os
import glob
import numpy as np
from tqdm.notebook import tqdm
from shutil import copyfile
from datetime import datetime

# IMAGE IMPORTS
import cv2
from PIL import Image

# GIS IMPORTS
import fiona
import pyproj
from affine import Affine
from shapely.geometry import shape, mapping
from shapely.geometry import Point, LineString
from shapely.ops import transform, nearest_points, snap
import pandas as pd
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from scipy.spatial import cKDTree

# PLOTTING IMPORTS
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# CUSTOM UTILITIES
from WorldFileUtils import *
from GeometryUtils import *
from icp import *

Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
templates_dir = "data/templates/"
tempfiles_dir = "tempfiles/"

boundary_shapefile = f"{templates_dir}HCAD_Harris_County_Boundary.shp"
boundary_points    = f'{tempfiles_dir}boundary_points.shp'
roads_points       = f'{tempfiles_dir}roads_points.shp'

tile_file = r"data_local\000_WorkingFiles\HarrisIndexes\48201CIND0_0992.tif"

roads_fn = f"data/Roads/TexasHighways.shp"
selected_transform = f"{tempfiles_dir}SelectedTransform.png"

In [24]:
def extract_thick_lines(image_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, 0)
    image = np.where(np.asarray(image), 255, 0).astype(np.uint8)
    
    # Apply Gaussian blur to reduce noise
    image = cv2.GaussianBlur(image, (31, 31), 0)
    image = cv2.GaussianBlur(image, (31, 31), 0)
    image = cv2.GaussianBlur(image, (31, 31), 0)
    image = cv2.GaussianBlur(image, (15, 15), 0)

    # Perform adaptive thresholding to obtain a binary image
    # _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Define the structuring element for morphological operations
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    # Perform erosion and dilation to enhance the lines
    # eroded = cv2.erode(thresholded, kernel, iterations=4)
    image = cv2.dilate(image, np.ones((3,3), np.uint8), iterations=3)
    image = cv2.adaptiveThreshold(image, 255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,19, 8)
    
    return image

# Example usage
image_path = tile_file

result = extract_thick_lines(image_path)
cv2.imwrite(f"{tempfiles_dir}ExtractThickLines.png", result)

True

Here you fixed it up, now you need to flip it

In [13]:
data_dir = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data\TileLocator\out"
image_to_flip = cv2.imread(f"{data_dir}/2.png")
image_to_flip = 255 - np.asarray(image_to_flip)[:,:,0]
cv2.imwrite(f"{data_dir}/2_1.png", image_to_flip)

True

In [12]:
image_to_flip.shape

(10200, 14840)